In [1]:
import pandas as pd
import ssl
import matplotlib.pyplot as plt
import numpy as np
import hashlib
import boto3

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
ssl._create_default_https_context = ssl._create_unverified_context

# Tooling

### Upload S3

In [5]:
def UploadS3(filepath, key):
    s3 = boto3.client("s3")
    bucket_name = "economia-popular-delivery-content-indices"
    s3.upload_file(
        Filename=filepath,
        Bucket=bucket_name,
        Key=key,
        ExtraArgs={
            "ACL": "public-read",
            "ServerSideEncryption": "AES256"
        }
    )

    return True

### Hash String

In [6]:
def create_hash(text):
    return hashlib.sha256(text.encode()).hexdigest()

### IBGE parser

In [7]:
def ibge_dataframe_builder(df, items, classification, year):
    year = "{}".format(year)
    for item in items:
        dimensao = item["localidade"]["nivel"]["nome"]
        localidade = item["localidade"]["nome"]
        populacao = item["serie"]["{}".format(year)]
        id = create_hash(year + "-" + dimensao + "-" + localidade)
        
        row = {
            "id": id,
            "ano": year,
            "dimensao": dimensao,
            "localidade": localidade,
            "cor_raca": classification, 
            "populacao": populacao
        }
        df = df.append(row, ignore_index=True)        
    return df

## Dataframes

### Criando a estrutura base do Dataframe Vazia

In [8]:
columns = ["id", "ano", "dimensao", "localidade" , "cor_raca", "populacao"]
df = pd.DataFrame(columns=columns)
print(df)

Empty DataFrame
Columns: [id, ano, dimensao, localidade, cor_raca, populacao]
Index: []


In [9]:
columns_consolidado = ["id", "ano", "dimensao", "localidade" , "populacao_branca", "populacao_negra", "populacao_amarela", "populacao_parda","populacao_indigena" ,"populacao_sem_declaracao" ,"populacao_total"]
df_consolidado = pd.DataFrame(columns=columns_consolidado)
print(df_consolidado)

Empty DataFrame
Columns: [id, ano, dimensao, localidade, populacao_branca, populacao_negra, populacao_amarela, populacao_parda, populacao_indigena, populacao_sem_declaracao, populacao_total]
Index: []


In [10]:
df.head()

,id,ano,dimensao,localidade,cor_raca,populacao


In [11]:
df.tail()

,id,ano,dimensao,localidade,cor_raca,populacao


In [12]:
df.shape

(0, 6)

### URL's de Consulta no IBGE - Brasil

In [13]:
brasil_total_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[0]|12056[99566]"
brasil_branca_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[2776]|12056[99566]"
brasil_preta_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[2777]|12056[99566]"
brasil_amarela_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[2778]|12056[99566]"
brasil_parda_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[2779]|12056[99566]"
brasil_indigena_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[2780]|12056[99566]"
brasil_sem_declaracao_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[2781]|12056[99566]"


### URL's de Consulta no IBGE - Região Metropolitanas

In [14]:
brasil_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N1[all]&classificacao=86[all]|12056[99566]"
total_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[0]|12056[99566]|58[0]"
branca_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2776]|12056[99566]|58[0]"
preta_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2777]|12056[99566]|58[0]"
amarela_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2778]|12056[99566]|58[0]" 
parda_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2779]|12056[99566]|58[0]"
indigena_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2780]|12056[99566]|58[0]"
sem_declaracao_url = "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N7[all]|N3[all]&classificacao=86[2781]|12056[99566]|58[0]"

## URL's de Consulta no IBGE - Municipios

In [15]:
urls_municipios = [
    {
        "cor_raca": "total",
        "url": "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N6[all]&classificacao=86[0]|12056[99566]"
    },    
    {
        "cor_raca": "branca",
        "url": "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N6[all]&classificacao=86[2776]|12056[99566]"
    },
    {
        "cor_raca": "preta",
        "url": "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N6[all]&classificacao=86[2777]|12056[99566]"
    },
    {
        "cor_raca": "amarela",
        "url": "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N6[all]&classificacao=86[2778]|12056[99566]"
    },
    {
        "cor_raca": "parda",
        "url": "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N6[all]&classificacao=86[2779]|12056[99566]"
    },
    {
        "cor_raca": "indigena",
        "url": "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N6[all]&classificacao=86[2780]|12056[99566]"
    },
    {
        "cor_raca": "sem_declaracao",
        "url": "https://servicodados.ibge.gov.br/api/v3/agregados/2098/periodos/2010/variaveis/140?localidades=N6[all]&classificacao=86[2781]|12056[99566]"
    }
]

In [16]:
for i in urls_municipios:
    df_temp = pd.read_json(i['url'])
    series = df_temp.resultados[0][0]['series']
    df = ibge_dataframe_builder(df, series, i['cor_raca'], 2010)


In [17]:
df.shape

(38955, 6)

### População Brasil - Total

In [18]:
df_temp = pd.read_json(brasil_total_url)

In [19]:
series = df_temp.resultados[0][0]['series']

In [20]:
df = ibge_dataframe_builder(df, series, "total", 2010)

### População Brasil - Branca

In [21]:
df_temp = pd.read_json(brasil_branca_url)

In [22]:
series = df_temp.resultados[0][0]['series']

In [23]:
df = ibge_dataframe_builder(df, series, "branca", 2010)

### População brasil - Preta

In [24]:
df_temp = pd.read_json(brasil_preta_url)

In [25]:
series = df_temp.resultados[0][0]['series']

In [26]:
df = ibge_dataframe_builder(df, series, "preta", 2010)

### População Brasil - Amarela

In [27]:
df_temp = pd.read_json(brasil_amarela_url)

In [28]:
series = df_temp.resultados[0][0]['series']

In [29]:
df = ibge_dataframe_builder(df, series, "amarela", 2010)

### População Brasil - Parda

In [30]:
df_temp = pd.read_json(brasil_parda_url)

In [31]:
series = df_temp.resultados[0][0]['series']

In [32]:
df = ibge_dataframe_builder(df, series, "parda", 2010)

### População Brasileira - Indigena

In [33]:
df_temp = pd.read_json(brasil_indigena_url)

In [34]:
series = df_temp.resultados[0][0]['series']

In [35]:
df = ibge_dataframe_builder(df, series, "indigena", 2010)

### População Brasileira - Sem Declaração

In [36]:
df_temp = pd.read_json(brasil_sem_declaracao_url)

In [37]:
series = df_temp.resultados[0][0]['series']

In [38]:
df = ibge_dataframe_builder(df, series, "sem_declaracao", 2010)

### População Total

In [39]:
df_temp = pd.read_json(total_url)

In [40]:
series = df_temp.resultados[0][0]['series']

In [41]:
df = ibge_dataframe_builder(df, series, "total", 2010)

In [42]:
df.shape

(39025, 6)

### População Declarada como Branca

In [43]:
df_temp = pd.read_json(branca_url)

In [44]:
series = df_temp.resultados[0][0]['series']

In [45]:
df = ibge_dataframe_builder(df, series, "branca", 2010)

In [46]:
df.shape

(39088, 6)

### População declarada como preta

In [47]:
df_temp = pd.read_json(preta_url)

In [48]:
series = df_temp.resultados[0][0]['series']

In [49]:
df = ibge_dataframe_builder(df, series, "preta", 2010)

In [50]:
df.shape

(39151, 6)

### Populacão declarada como amarela

In [51]:
df_temp = pd.read_json(amarela_url)

In [52]:
series = df_temp.resultados[0][0]['series']

In [53]:
df = ibge_dataframe_builder(df, series, "amarela", 2010)

In [54]:
df.shape

(39214, 6)

### População Parda

In [55]:
df_temp = pd.read_json(parda_url)

In [56]:
series = df_temp.resultados[0][0]['series']

In [57]:
df = ibge_dataframe_builder(df, series, "parda", 2010)

In [58]:
df.shape

(39277, 6)

### População Indigena

In [59]:
df_temp = pd.read_json(indigena_url)

In [60]:
series = df_temp.resultados[0][0]['series']

In [61]:
df = ibge_dataframe_builder(df, series, "indigena", 2010)

In [62]:
df.shape

(39340, 6)

### População Sem Declaração

In [63]:
df_temp = pd.read_json(sem_declaracao_url)

In [64]:
series = df_temp.resultados[0][0]['series']

In [65]:
df = ibge_dataframe_builder(df, series, "sem_declaracao", 2010)

In [66]:
df.shape

(39403, 6)

## Sumário Exploratório

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39403 entries, 0 to 39402
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          39403 non-null  object
 1   ano         39403 non-null  object
 2   dimensao    39403 non-null  object
 3   localidade  39403 non-null  object
 4   cor_raca    39403 non-null  object
 5   populacao   39403 non-null  object
dtypes: object(6)
memory usage: 1.8+ MB


In [68]:
df.head()

,id,ano,dimensao,localidade,cor_raca,populacao
0,1c1d9ffe0dc859b88aa264e0b98f2587562dd3c4b869f6...,2010,Município,Alta Floresta D'Oeste - RO,total,10526
1,c3f4fa4194f59ad76eb4f090dd611e4d2e52d94a0fde16...,2010,Município,Ariquemes - RO,total,45162
2,d9f3c32a42d979735fc4326a48e9b2dac7d40a953094be...,2010,Município,Cabixi - RO,total,2867
3,5179d62c8be2ff98348c6c48e88cd3492257376ebb205e...,2010,Município,Cacoal - RO,total,39977
4,71b58e4bdc367b0fdd83866ec7a6f71c2eb9d6d07aaa68...,2010,Município,Cerejeiras - RO,total,8670


In [69]:
df.tail()

,id,ano,dimensao,localidade,cor_raca,populacao
39398,d23860d67f080e8454155455c2cc47c34f2637b24ba27f...,2010,Unidade da Federação,Rio Grande do Sul,sem_declaracao,34
39399,29cc530b8162c0ff5f37a34a07dc7f2dc71e212ef73917...,2010,Unidade da Federação,Mato Grosso do Sul,sem_declaracao,-
39400,f9a89474f5ba626244505fa48a4e9bbf243418ecfbe0ff...,2010,Unidade da Federação,Mato Grosso,sem_declaracao,-
39401,3991c5a6b9fc6bf8aac55aa0cd131e551d2a6e4405ac8e...,2010,Unidade da Federação,Goiás,sem_declaracao,13
39402,be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc46870...,2010,Unidade da Federação,Distrito Federal,sem_declaracao,-


In [70]:
df.cor_raca.value_counts()

total             5629
branca            5629
preta             5629
amarela           5629
parda             5629
indigena          5629
sem_declaracao    5629
Name: cor_raca, dtype: int64

In [71]:
df.id.value_counts()

1c1d9ffe0dc859b88aa264e0b98f2587562dd3c4b869f681fb0feb60251cd096    7
ce5460e921414b2a454d4ee1f21a094ef636982e7329eca74293b40370511f7b    7
2a764aea165ebac6a543388f8f13f7caa6847c84a4ef5eed95c2930751ccf301    7
cd0b4269af600af94c4b2cd23304160b1fcf6f0ac11387e334dbeed4fedc2908    7
b9b7a930f7ece9d79871cb75d9dd3b1c3db5ebe0b380aac066ce242180d4598e    7
                                                                   ..
82877bb55ebe1742e03e92997568ec3b00778c26a4db7d4f3044aade06740afd    7
57dd92bc4cc5f771f00f347255fb460d38d0a11637eb6c2e8a27897627fba688    7
27d045fe3614944469412f1137ff60b00e001f7c53ccbd487a402871cf23d5f8    7
c499e2e1bf8803e13c401e6e6d36f30997fa201a435ec443e4bd0c1bc014d68d    7
be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc468702a63bb49b13a836fb6    7
Name: id, Length: 5629, dtype: int64

In [72]:
df.replace("-", 0, inplace=True)

In [73]:
df.replace("...", 0, inplace=True)

# Consolidação dos Dados

## Criando o DF de referencia

In [74]:
df_consolidado.id = df[["id"]].copy()

In [75]:
df_consolidado.head()

,id,ano,dimensao,localidade,populacao_branca,populacao_negra,populacao_amarela,populacao_parda,populacao_indigena,populacao_sem_declaracao,populacao_total
0,1c1d9ffe0dc859b88aa264e0b98f2587562dd3c4b869f6...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c3f4fa4194f59ad76eb4f090dd611e4d2e52d94a0fde16...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,d9f3c32a42d979735fc4326a48e9b2dac7d40a953094be...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5179d62c8be2ff98348c6c48e88cd3492257376ebb205e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,71b58e4bdc367b0fdd83866ec7a6f71c2eb9d6d07aaa68...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
df_consolidado.id.value_counts()

1c1d9ffe0dc859b88aa264e0b98f2587562dd3c4b869f681fb0feb60251cd096    7
ce5460e921414b2a454d4ee1f21a094ef636982e7329eca74293b40370511f7b    7
2a764aea165ebac6a543388f8f13f7caa6847c84a4ef5eed95c2930751ccf301    7
cd0b4269af600af94c4b2cd23304160b1fcf6f0ac11387e334dbeed4fedc2908    7
b9b7a930f7ece9d79871cb75d9dd3b1c3db5ebe0b380aac066ce242180d4598e    7
                                                                   ..
82877bb55ebe1742e03e92997568ec3b00778c26a4db7d4f3044aade06740afd    7
57dd92bc4cc5f771f00f347255fb460d38d0a11637eb6c2e8a27897627fba688    7
27d045fe3614944469412f1137ff60b00e001f7c53ccbd487a402871cf23d5f8    7
c499e2e1bf8803e13c401e6e6d36f30997fa201a435ec443e4bd0c1bc014d68d    7
be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc468702a63bb49b13a836fb6    7
Name: id, Length: 5629, dtype: int64

In [77]:
df_consolidado = df_consolidado.drop_duplicates(subset='id', keep='first')

In [78]:
df_consolidado.id.value_counts()

1c1d9ffe0dc859b88aa264e0b98f2587562dd3c4b869f681fb0feb60251cd096    1
ce5460e921414b2a454d4ee1f21a094ef636982e7329eca74293b40370511f7b    1
2a764aea165ebac6a543388f8f13f7caa6847c84a4ef5eed95c2930751ccf301    1
cd0b4269af600af94c4b2cd23304160b1fcf6f0ac11387e334dbeed4fedc2908    1
b9b7a930f7ece9d79871cb75d9dd3b1c3db5ebe0b380aac066ce242180d4598e    1
                                                                   ..
82877bb55ebe1742e03e92997568ec3b00778c26a4db7d4f3044aade06740afd    1
57dd92bc4cc5f771f00f347255fb460d38d0a11637eb6c2e8a27897627fba688    1
27d045fe3614944469412f1137ff60b00e001f7c53ccbd487a402871cf23d5f8    1
c499e2e1bf8803e13c401e6e6d36f30997fa201a435ec443e4bd0c1bc014d68d    1
be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc468702a63bb49b13a836fb6    1
Name: id, Length: 5629, dtype: int64

In [79]:
df_consolidado.index = df_consolidado.id

In [80]:
df_consolidado = df_consolidado.drop(columns="id", axis=1)

In [81]:
df_consolidado.head()

,ano,dimensao,localidade,populacao_branca,populacao_negra,populacao_amarela,populacao_parda,populacao_indigena,populacao_sem_declaracao,populacao_total
id,,,,,,,,,,
1c1d9ffe0dc859b88aa264e0b98f2587562dd3c4b869f681fb0feb60251cd096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c3f4fa4194f59ad76eb4f090dd611e4d2e52d94a0fde167dfff98d2ca61605f9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
d9f3c32a42d979735fc4326a48e9b2dac7d40a953094be96c1303606d4449d94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5179d62c8be2ff98348c6c48e88cd3492257376ebb205e452964980cf34bc1c4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71b58e4bdc367b0fdd83866ec7a6f71c2eb9d6d07aaa688b851944936fb80bfc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Inputando os dados quantitativos da população

In [82]:
for index, row in df.iterrows():
    # print(row.id)
    df_consolidado.loc[row.id, "ano"] = row.ano
    df_consolidado.loc[row.id, "dimensao"] = row.dimensao
    df_consolidado.loc[row.id, "localidade"] = row.localidade

    if row.cor_raca == "total":
        df_consolidado.loc[row.id, "populacao_total"] = row.populacao
    elif row.cor_raca == "branca":
        df_consolidado.loc[row.id, "populacao_branca"] = row.populacao
    elif row.cor_raca == "preta":
        df_consolidado.loc[row.id, "populacao_negra"] = row.populacao
    elif row.cor_raca == "amarela":
        df_consolidado.loc[row.id, "populacao_amarela"] = row.populacao    
    elif row.cor_raca == "parda":
        df_consolidado.loc[row.id, "populacao_parda"] = row.populacao 
    elif row.cor_raca == "indigena":
        df_consolidado.loc[row.id, "populacao_indigena"] = row.populacao
    elif row.cor_raca == "sem_declaracao":
        df_consolidado.loc[row.id, "populacao_sem_declaracao"] = row.populacao 

In [83]:
df_consolidado.replace("-", 0, inplace=True)

In [84]:
df_consolidado.replace("...", 0, inplace=True)

In [85]:
df_consolidado.replace(np.nan, 0, inplace=True)

In [86]:
df_consolidado["populacao_total"]

id
1c1d9ffe0dc859b88aa264e0b98f2587562dd3c4b869f681fb0feb60251cd096      10526
c3f4fa4194f59ad76eb4f090dd611e4d2e52d94a0fde167dfff98d2ca61605f9      45162
d9f3c32a42d979735fc4326a48e9b2dac7d40a953094be96c1303606d4449d94       2867
5179d62c8be2ff98348c6c48e88cd3492257376ebb205e452964980cf34bc1c4      39977
71b58e4bdc367b0fdd83866ec7a6f71c2eb9d6d07aaa688b851944936fb80bfc       8670
                                                                     ...   
d23860d67f080e8454155455c2cc47c34f2637b24ba27fffad30c2a1450a61a1    5818585
29cc530b8162c0ff5f37a34a07dc7f2dc71e212ef73917292046c296cdf73190    1258710
f9a89474f5ba626244505fa48a4e9bbf243418ecfbe0ff54873a035a978f5f37    1545518
3991c5a6b9fc6bf8aac55aa0cd131e551d2a6e4405ac8e7f18333334f80d1da1    3158254
be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc468702a63bb49b13a836fb6    1402349
Name: populacao_total, Length: 5629, dtype: object

In [87]:
df_consolidado.head()

,ano,dimensao,localidade,populacao_branca,populacao_negra,populacao_amarela,populacao_parda,populacao_indigena,populacao_sem_declaracao,populacao_total
id,,,,,,,,,,
1c1d9ffe0dc859b88aa264e0b98f2587562dd3c4b869f681fb0feb60251cd096,2010,Município,Alta Floresta D'Oeste - RO,4674,571,65,5150,67,0,10526
c3f4fa4194f59ad76eb4f090dd611e4d2e52d94a0fde167dfff98d2ca61605f9,2010,Município,Ariquemes - RO,16057,4644,976,23418,66,0,45162
d9f3c32a42d979735fc4326a48e9b2dac7d40a953094be96c1303606d4449d94,2010,Município,Cabixi - RO,1446,170,32,1216,3,0,2867
5179d62c8be2ff98348c6c48e88cd3492257376ebb205e452964980cf34bc1c4,2010,Município,Cacoal - RO,16979,2752,510,19207,530,0,39977
71b58e4bdc367b0fdd83866ec7a6f71c2eb9d6d07aaa688b851944936fb80bfc,2010,Município,Cerejeiras - RO,4269,543,104,3740,14,0,8670


In [88]:
df_consolidado.tail()

,ano,dimensao,localidade,populacao_branca,populacao_negra,populacao_amarela,populacao_parda,populacao_indigena,populacao_sem_declaracao,populacao_total
id,,,,,,,,,,
d23860d67f080e8454155455c2cc47c34f2637b24ba27fffad30c2a1450a61a1,2010,Unidade da Federação,Rio Grande do Sul,4900565,313561,19370,571704,13351,34,5818585
29cc530b8162c0ff5f37a34a07dc7f2dc71e212ef73917292046c296cdf73190,2010,Unidade da Federação,Mato Grosso do Sul,598609,72722,15840,551750,19789,0,1258710
f9a89474f5ba626244505fa48a4e9bbf243418ecfbe0ff54873a035a978f5f37,2010,Unidade da Federação,Mato Grosso,596556,131222,19135,787210,11395,0,1545518
3991c5a6b9fc6bf8aac55aa0cd131e551d2a6e4405ac8e7f18333334f80d1da1,2010,Unidade da Federação,Goiás,1299904,232175,56025,1565193,4943,13,3158254
be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc468702a63bb49b13a836fb6,2010,Unidade da Federação,Distrito Federal,579838,121401,24868,672191,4049,0,1402349


### Extração do estado

In [89]:
df_consolidado['estado'] = df_consolidado['localidade'].str.extract(' - (\w\w)$')

In [90]:
df_consolidado.estado.value_counts()

MG    855
SP    648
RS    497
BA    418
PR    402
SC    301
GO    247
PB    225
PI    224
MA    219
CE    186
PE    186
RN    168
PA    144
MT    142
TO    139
AL    104
RJ     93
ES     79
MS     78
SE     76
AM     63
RO     52
AC     22
AP     17
RR     15
DF      1
Name: estado, dtype: int64

In [91]:
df_consolidado.localidade.value_counts()

Rio de Janeiro - RJ        2
Florianópolis - SC         2
Maceió - AL                2
João Pessoa - PB           2
Campina Grande - PB        2
                          ..
Bom Jesus da Lapa - BA     1
Boa Vista do Tupim - BA    1
Boa Nova - BA              1
Biritinga - BA             1
Distrito Federal           1
Name: localidade, Length: 5605, dtype: int64

In [92]:
df_consolidado['localidade'] = df_consolidado['localidade'].str.replace(' - \w\w$', '', regex=True)

In [93]:
df_consolidado.localidade.value_counts()

São Domingos        5
Bom Jesus           5
Bonito              4
Planalto            4
Santa Terezinha     4
                   ..
Dom Macedo Costa    1
Dom Basílio         1
Dias d'Ávila        1
Dário Meira         1
Distrito Federal    1
Name: localidade, Length: 5324, dtype: int64

### Convertendo as colunas para integer

In [94]:
df_consolidado["populacao_total"] = df_consolidado["populacao_total"].astype(int)
df_consolidado["populacao_branca"] = df_consolidado["populacao_branca"].astype(int)
df_consolidado["populacao_negra"] = df_consolidado["populacao_negra"].astype(int)
df_consolidado["populacao_amarela"] = df_consolidado["populacao_amarela"].astype(int)
df_consolidado["populacao_parda"] = df_consolidado["populacao_parda"].astype(int)
df_consolidado["populacao_indigena"] = df_consolidado["populacao_indigena"].astype(int)
df_consolidado["populacao_sem_declaracao"] = df_consolidado["populacao_sem_declaracao"].astype(int)

### Conferencia da Consolidação Quantitativa

In [95]:
df_consolidado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5629 entries, 1c1d9ffe0dc859b88aa264e0b98f2587562dd3c4b869f681fb0feb60251cd096 to be5a3f4e95d5b5c829f8a46a27cafba1d52ef88bc468702a63bb49b13a836fb6
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ano                       5629 non-null   object
 1   dimensao                  5629 non-null   object
 2   localidade                5629 non-null   object
 3   populacao_branca          5629 non-null   int64 
 4   populacao_negra           5629 non-null   int64 
 5   populacao_amarela         5629 non-null   int64 
 6   populacao_parda           5629 non-null   int64 
 7   populacao_indigena        5629 non-null   int64 
 8   populacao_sem_declaracao  5629 non-null   int64 
 9   populacao_total           5629 non-null   int64 
 10  estado                    5601 non-null   object
dtypes: int64(7), object(4)
memory usage: 656.8+ KB


In [96]:
df_consolidado.iloc[0]

ano                                          2010
dimensao                                Município
localidade                  Alta Floresta D'Oeste
populacao_branca                             4674
populacao_negra                               571
populacao_amarela                              65
populacao_parda                              5150
populacao_indigena                             67
populacao_sem_declaracao                        0
populacao_total                             10526
estado                                         RO
Name: 1c1d9ffe0dc859b88aa264e0b98f2587562dd3c4b869f681fb0feb60251cd096, dtype: object

In [97]:
df_consolidado.iloc[1]

ano                              2010
dimensao                    Município
localidade                  Ariquemes
populacao_branca                16057
populacao_negra                  4644
populacao_amarela                 976
populacao_parda                 23418
populacao_indigena                 66
populacao_sem_declaracao            0
populacao_total                 45162
estado                             RO
Name: c3f4fa4194f59ad76eb4f090dd611e4d2e52d94a0fde167dfff98d2ca61605f9, dtype: object

## Criando as porcentagens

In [98]:
df_consolidado['populacao_branca_porcentagem'] = ((df_consolidado['populacao_branca'] / df_consolidado['populacao_total']) * 100).round(2) 

In [99]:
df_consolidado['populacao_negra_porcentagem'] = ((df_consolidado['populacao_negra'] / df_consolidado['populacao_total']) * 100).round(2) 

In [100]:
df_consolidado['populacao_amarela_porcentagem'] = ((df_consolidado['populacao_amarela'] / df_consolidado['populacao_total']) * 100).round(2) 

In [101]:
df_consolidado['populacao_parda_porcentagem'] = ((df_consolidado['populacao_parda'] / df_consolidado['populacao_total']) * 100).round(2) 

In [102]:
df_consolidado['populacao_indigena_porcentagem'] = ((df_consolidado['populacao_indigena'] / df_consolidado['populacao_total']) * 100).round(2) 

In [103]:
df_consolidado['populacao_sem_declaracao_porcentagem'] = ((df_consolidado['populacao_sem_declaracao'] / df_consolidado['populacao_total']) * 100).round(2) 

In [104]:
df_consolidado.iloc[1]

ano                                          2010
dimensao                                Município
localidade                              Ariquemes
populacao_branca                            16057
populacao_negra                              4644
populacao_amarela                             976
populacao_parda                             23418
populacao_indigena                             66
populacao_sem_declaracao                        0
populacao_total                             45162
estado                                         RO
populacao_branca_porcentagem                35.55
populacao_negra_porcentagem                 10.28
populacao_amarela_porcentagem                2.16
populacao_parda_porcentagem                 51.85
populacao_indigena_porcentagem               0.15
populacao_sem_declaracao_porcentagem          0.0
Name: c3f4fa4194f59ad76eb4f090dd611e4d2e52d94a0fde167dfff98d2ca61605f9, dtype: object

In [105]:
df_consolidado.dimensao.value_counts()

Município               5565
Região Metropolitana      36
Unidade da Federação      27
Brasil                     1
Name: dimensao, dtype: int64

# Upload

## Salvando os Dados

In [106]:
df_consolidado.to_json('datasets/pessoas_economicamente_ativas.json', orient='records', lines=True)

In [107]:
df_consolidado.to_csv('datasets/pessoas_economicamente_ativas.csv', index=False)

In [111]:
UploadS3('datasets/pessoas_economicamente_ativas.json', 'sociais/pessoas_economicamente_ativas.pessoas_economicamente_ativas.json')

True

In [110]:
UploadS3('datasets/pessoas_economicamente_ativas.csv', 'sociais/pessoas_economicamente_ativas/pessoas_economicamente_ativas.csv')

True